<a href="https://colab.research.google.com/github/JHyunjun/SuperResolution/blob/main/231014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created By unknown-user(Hyundai CTO Dacon)
# From 23.10.14

In [ ]:
# Importing
import tensorflow as tf
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Lambda, Input, Conv2D, Add
from tensorflow.keras.models import Model
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import median_filter, uniform_filter


In [ ]:
# Salt and pepper Denoising

def selective_median_filter(img, kernel_size=3, threshold1 = 0, threshold2 = 1):
    # Masks where the image has values close to 0 or 1
    mask_salt = img > (1.0 - threshold1) # white
    mask_pepper = img < threshold2 # black
    mask_combined = tf.math.logical_or(mask_salt, mask_pepper).numpy()

    # Convert image to numpy for processing
    img_numpy = img.numpy()

    # Apply median filter only where the mask is true
    for i in range(img_numpy.shape[2]):  # For each channel
        channel = img_numpy[:,:,i]
        channel_filtered = median_filter(channel, size=kernel_size)
        #channel_filtered = uniform_filter(channel, size=kernel_size)
        # Replace only the noisy pixels
        channel[mask_combined[:,:,0]] = channel_filtered[mask_combined[:,:,0]]
        img_numpy[:,:,i] = channel

    return tf.convert_to_tensor(img_numpy, dtype=tf.float32)

In [ ]:
# Data
directory_path_train_lr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_lr"
directory_path_train_hr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_hr"

import tensorflow as tf
import os

batch_size = 32
image_slicing = 500

lr_image_files = sorted([os.path.join(directory_path_train_lr, f) for f in os.listdir(directory_path_train_lr)])
hr_image_files = sorted([os.path.join(directory_path_train_hr, f) for f in os.listdir(directory_path_train_hr)])

lr_image_files_3000 = lr_image_files[:image_slicing]
hr_image_files_3000 = hr_image_files[:image_slicing]

# pre-processing
def load_images(lr_path, hr_path):
    lr = tf.io.read_file(lr_path)
    lr = tf.image.decode_jpeg(lr, channels=3)
    lr = tf.image.resize(lr, [256, 256])
    lr = lr / 255.0
    lr_filter = tf.py_function(func=selective_median_filter, inp=[lr], Tout=tf.float32)

    hr = tf.io.read_file(hr_path)
    hr = tf.image.decode_jpeg(hr, channels=3)
    hr = tf.image.resize(hr, [1024, 1024])
    hr = hr / 255.0

    return lr, hr, lr_filter

train_dataset = tf.data.Dataset.from_tensor_slices((lr_image_files_3000, hr_image_files_3000))
train_dataset = train_dataset.map(load_images)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
def show_images(dataset, num_samples=3):
    for i, (lr_image, hr_image, lr_filter) in enumerate(dataset.unbatch().take(num_samples)):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(lr_image.numpy())
        plt.title(f"Low Resolution Image {i+1}")

        plt.subplot(1, 3, 2)
        plt.imshow(lr_filter.numpy())
        plt.title(f"Filtering Image {i+1}")

        plt.subplot(1, 3, 3)
        plt.imshow(hr_image.numpy())
        plt.title(f"High Resolution Image {i+1}")

        plt.show()

show_images(train_dataset)

In [ ]:
#Super Resolution

def residual_block(x, filters, kernel_size=3, scaling=0.1):
    tmp = Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    tmp = Conv2D(filters, kernel_size, padding='same')(tmp)
    tmp = Add()([x, tmp])
    return tmp

def upsample(x, scale=4):
    return Lambda(lambda x: tf.nn.depth_to_space(x, scale))(x)

def EDSR(input_shape=(256,256,3), scale=4):
    inputs = Input(input_shape)

    x = Conv2D(64, 9, padding='same', activation='relu')(inputs)
    for _ in range(5):  # Number of residual blocks
        x = residual_block(x, 64)

    x = Conv2D(64, 3, padding='same')(x)
    x = upsample(x, scale)
    outputs = Conv2D(3, 9, padding='same')(x)

    return Model(inputs, outputs)

model = EDSR()
model.compile(optimizer='adam', loss='mse')
model.summary()

model.fit(train_dataset, epochs=20)

In [ ]:
test_lr_paths = lr_image_files_3000[:3]
test_hr_paths = hr_image_files_3000[:3]


for test_lr_path, test_hr_path in zip(test_lr_paths, test_hr_paths):

    lr, hr = load_images(test_lr_path, test_hr_path)
    lr_input = tf.expand_dims(lr, 0)

    sr = model.predict(lr_input)
    sr = np.clip(sr, 0, 1)

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(tf.squeeze(lr))
    plt.title("LR Image")

    plt.subplot(1, 3, 2)
    plt.imshow(tf.squeeze(sr))
    plt.title("SR (Model Output) Image")

    plt.subplot(1, 3, 3)
    plt.imshow(tf.squeeze(hr))
    plt.title("HR Image")

    plt.show()
